In [1]:
%matplotlib inline
from CartPole import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

# Task 3

## Introduce noise to observed variables

In [7]:
def add_noise(data_array, var=1):
    if type(data_array) == list: data_array = np.array(data_array)
            
    noisy_array = np.random.normal(0,var,(data_array.shape))*data_array
    return noisy_array

In [26]:
plot_path = 'plots/Task3/'
index_to_variable = {0 : 'cart_location', 1 : 'cart_velocity', 2 : 'pole_angle', 3 : 'pole_velocity', 4 : 'action'}
index_pair_list = [[0,1],[0,2],[0,3],[1,2],[1,3],[2,3],[0,4],[1,4],[2,4],[3,4]]
no_samples = 20
x_range_for_index = {0 : np.linspace(-5,5,no_samples), 1 : np.linspace(-10,10,no_samples), 
                     2 : np.linspace(-np.pi,np.pi,no_samples), 3 : np.linspace(-15,15,no_samples), 
                     4 : np.linspace(-20,20,no_samples)}

n = 2000
no_RBC = 320 
lam = 0.00001
train_proportion = 0.8
x,y = generate_data(n)
    
x_train, y_train, x_test, y_test = x[:int(n*train_proportion)], y[:int(n*train_proportion)], x[int(n*train_proportion):], y[int(n*train_proportion):]
sigma = np.std(x_train, axis=0)

# alpha, X_i_vals = train_alpha(x_train, y_train, no_RBC, sigma, n, train_proportion)
# predictions = predict(x_test, alpha, X_i_vals, sigma)